# Michelson kernel basics
Welcome, friend! This an ultimate Michelson playground, and in this tutorial, you will learn how to make the most of all available functionality.  
If there are any questions, please ask in our telegram chat https://t.me/MavrykNetwork

In [ ]:
PUSH string "Hello, world!"

Michelson kernel is built on top of a custom interpreter which does not typecheck the whole script before execution but at runtime instead. Also, it allows developer to check the stack state at any time and for arbitrary depth.  
This enables a step-by-step coding approach which can save time while learning language or making a prototype or demo.

In [ ]:
DROP

## Context, stack, and notebook cells
When you start a kernel, a new instance of context is initialized. It stores the Michelson stack, stub values for the blockchain-specific instructions (e.g. `BALANCE` or `SENDER`), big map pool, origination index, and some internal variables.  
This context is shared across all cells. Note, that the cell's position doesn't matter, only the execution order.

In [ ]:
BALANCE  # balance is initialized with a default value

When you execute a cell, messages can appear in the `stdout` and` stderr` streams, as well as the optional result at the end.  
Let's try to execute a sequence of commands:

In [ ]:
PUSH mumav 0 ;
IFCMPEQ { FAIL } { PUSH string "We got money!" }

When there's more than one command a verbose logging is enabled. It can be disabled which will be shown a bit later.  
If the latest command in the sequences pushes an item onto the stack - it will be returned as a result.

### Inspecting the stack

In [ ]:
DEBUG False  # we just have disabled the verbose output

In [ ]:
PUSH (list string) { "a" ; "b" ; "c" } ;
PRINT "{0} is on top of the stack, then goes {1}" ;  # still we can printf anything to stdout
CONCAT @abc

Time to inspect the stack, there is `DUMP` helper for that. You can use it with or without depth specified (all elements)

In [ ]:
DUMP 2

In [ ]:
DROP_ALL ; DUMP

### Errors
if any instruction in a cell has failed the whole context is rolled back to the previous state. So you don't have to rerun everything from the beginning. Here are a few examples of possible errors:

In [ ]:
ADD

In [ ]:
HELLO

In [ ]:
PUSH mumav 1 ; EQ

In [ ]:
DEBUG True  # and we continue to the next topic ^_^

## Blockchain-specific instructions
First of all, there are several instructions that in a real environment push some value from the execution context, as `AMOUNT`, `SENDER`, `SOURCE`, `BALANCE`, etc. Here we are detached from any particular chain, but you have an opportunity to patch these values:

In [ ]:
PATCH AMOUNT 100500 ; AMOUNT

In [ ]:
PATCH AMOUNT ; AMOUNT  # still have default value 0

### Internal operations
Despite internal operations will never apply, Michelson kernel tries to emulate the standard behavior as closely as possible.

In [ ]:
UNIT;                    # starting storage for contract
AMOUNT;                  # Push the starting balance
NONE key_hash;           # No delegate
CREATE_CONTRACT          # Create the contract
{ parameter unit ;
  storage unit ;
  code { FAIL } };

### Script sections
`parameter`, `storage`, and `code` instructions are supported as well, what they do is basically store the argument in the context.

In [ ]:
parameter unit ;
storage string ;
code { DROP ; PUSH string "Hey!"; NIL operation ; PAIR }

In order to run this contract with particular parameters and initial storage you can use `RUN` helper:

In [ ]:
RUN %default Unit "hi"  # %default is entrypoint

`RUN` returns storage, big map diff (if applicable), and list of internal operations.  
You can also load contract from file or chain via `INCLUDE` helper:

In [ ]:
INCLUDE "mainnet:KT1VG2WtYdSWz5E7chTeAdDPZNy2MpP8pTfL"  # can also be a filename

### Step by step debugging
In case you don't want to execute the whole `code`, you can mark the beginning of the contract by calling `BEGIN` (with the same arguments as `RUN`) and in the end call `COMMIT`:

In [ ]:
BEGIN %refund 0xdeadbeef (Pair {} Unit)

In [ ]:
CDR ; NIL operation ; PAIR ; COMMIT

You can also break the current pseudo-exection by calling `RESET` - it will clear the stack and all the cached big maps:

In [ ]:
RESET

## Big maps
In the previous example, you might notice that we initialize the storage as an empty map, then it is displayed on the stack as -1, and as a result, it becomes 0.  
This is roughly how big map works in the real world: first, a temporary container is created, and if at the end of the contract execution it's still in use - a new big map is allocated. Basically big map is an integer pointer to a hashtable somewhere in the context.  
In our playground in order to check the big map state, you need to call `BIG_MAP_DIFF` helper.

In [ ]:
EMPTY_BIG_MAP string string

In [ ]:
PUSH string "two";
SOME;
PUSH string "one";
UPDATE

In [ ]:
BIG_MAP_DIFF  # works if the top item contains big maps

### Reusing big maps
So far, we have come across `alloc` and` update` actions in big map., but there are also `copy` and `remove` actions. We will need to pass one of the allocated big map pointer to another pseudo-contract:

In [ ]:
parameter unit;
storage (big_map int int);
code { CDR ; NIL operation ;PAIR }

In [ ]:
RUN Unit { Elt 1 2 ; Elt 2 3 }

In [ ]:
code { CDR ; PUSH int 5; SOME ; PUSH int 4; UPDATE ; NIL operation ; PAIR }

In [ ]:
RUN Unit 0  # passing previously allocated big_map #0

## Accessing on-chain data
Sometimes it is convenient to access the blockchain data right from the notebook. The `RESET` helper has an extra parameter that allows to specify the network we shoudl bind to.

In [ ]:
RESET "mainnet"

In [ ]:
CHAIN_ID ; NOW ; DUMP 2  # Few blockchain-specific instruction will change the behavior

In [ ]:
PUSH address "KT1UvfyLytrt71jh63YV4Yex5SmbNXpWHxtg" ; CONTRACT unit  # also, contract type checking is now working

### Real big maps
The coolest thing is that now you can access real big maps by a pointer, right from your pseudo-contract.  
If you are loading the contract source from the network, a special variable `Current` is initialized with the current contract storage.

In [ ]:
INCLUDE "KT1UvfyLytrt71jh63YV4Yex5SmbNXpWHxtg"

In [ ]:
BEGIN %setPause True STORAGE

In [ ]:
CDR ; CAR ; PUSH string "MeuJogo" ; GET